In [21]:
from neo4j import GraphDatabase
import folium
import geocoder
import requests
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from neo4j import GraphDatabase
import tkinter as tk

In [22]:
def split_coordinates(coord_string):
    # Diviser la chaîne par '|' pour obtenir chaque paire de coordonnées
    pairs = coord_string.split('|')
    
    # Initialiser une liste pour stocker les tuples de coordonnées
    coordinates = []
    
    # Pour chaque paire dans la liste des paires
    for pair in pairs:
        # Diviser la paire par '#' pour séparer latitude et longitude
        lat_long = pair.split('#')
        
        # Convertir les valeurs en float et les ajouter à la liste comme tuple
        if len(lat_long) == 2:
            try:
                longitude = float(lat_long[0])
                latitude = float(lat_long[1])
                if (latitude,longitude) not in coordinates :
                    coordinates.append((latitude,longitude))
            except ValueError:
                # Gérer le cas où les valeurs ne peuvent pas être converties en float
                print(f"Erreur de conversion pour la paire: {pair}")
    
    return coordinates


In [23]:
# Informations de connexion
uri = "bolt://localhost:7687"
user = "neo4j"
password = "bdsas2024"

# Connexion à Neo4j
# Initialisation du driver pour se connecter à la base de données Neo4j avec l'URI et les informations d'authentification fournies.
driver = GraphDatabase.driver(uri, auth=(user, password))

# Ouverture d'une session
# Ouverture d'une session pour exécuter les requêtes dans la base de données spécifiée, ici "ProjetNeo4j".
session = driver.session(database="ProjetNeo4j")

# Requête Cypher pour projeter un graphe pour les algorithmes de cheminement
query = '''
CALL gds.graph.project(
    'myGraph',  
    'Location',  
    'ROAD',  
    {
        relationshipProperties: 'length'  
    }
)
'''

# Exécution de la requête pour projeter le graphe
session.run(query)

# Fermeture de la session et du driver
# Fermeture de la session pour libérer les ressources.
session.close()

# Fermeture du driver pour libérer les ressources et terminer la connexion à la base de données.
driver.close()


In [24]:
from neo4j import GraphDatabase

def find_nearest_pharmacy(uri, user, password, latitude, longitude):
    # Connexion au driver Neo4j
    driver = GraphDatabase.driver(uri, auth=(user, password))
    try:
        # Ouverture de la session Neo4j
        with driver.session(database="ProjetNeo4j") as session:
            # Requête pour trouver le noeud le plus proche
            query = '''
            MATCH (n)
            WITH n,point.distance(point({latitude:''' + str(latitude) + ''', longitude:''' + str(longitude)+'''}), point({latitude: n.y, longitude: n.x})) AS dist
            RETURN n, n.osmid AS nearest_osmid, n.x AS nearest_latitude, n.y AS nearest_longitude
            ORDER BY dist
            LIMIT 1;
            '''
            # Exécution de la requête pour obtenir le noeud le plus proche
            nearest_result = session.run(query).single()
            if nearest_result is None:
                # Si aucun noeud n'est trouvé, renvoyer un message approprié
                return "No nodes found"
            
            nearest_osmid = nearest_result["nearest_osmid"]
            # Requête combinée pour trouver le chemin le plus court vers une pharmacie
            combined_query = f'''
            MATCH (source:Location {{osmid: {nearest_osmid}}})
            CALL gds.allShortestPaths.dijkstra.stream('myGraph', {{
                sourceNode: source,
                relationshipWeightProperty: 'length'
            }})
            YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
            WHERE gds.util.asNode(targetNode).pharmacy IS NOT NULL
            WITH
                index,
                gds.util.asNode(sourceNode).osmid AS sourceNodeName,
                gds.util.asNode(targetNode).osmid AS targetNodeName,
                totalCost,
                [nodeId IN nodeIds | gds.util.asNode(nodeId).osmid] AS nodeNames,
                costs,
                nodes(path) AS path,
                relationships(path) AS rels
            ORDER BY totalCost
            LIMIT 1
            WITH nodeNames, [rel IN rels | rel.geometry] AS geometries
            UNWIND range(0, size(nodeNames) - 2) AS idx
            MATCH (n1 {{osmid: nodeNames[idx]}})-[r]->(n2 {{osmid: nodeNames[idx + 1]}})
            RETURN collect(properties(r).geometry) AS relationshipProperties
            '''
            # Exécution de la requête combinée pour obtenir les propriétés des relations du chemin le plus court
            result = session.run(combined_query).single()
            if result is None:
                # Si aucun chemin vers une pharmacie n'est trouvé, renvoyer un message approprié
                return "No path to pharmacy found"
            
            # Récupération des propriétés des relations sous forme de chaîne de caractères
            relationship_properties = result["relationshipProperties"]
            chemin = ''.join(relationship_properties)
            # Retourner le chemin trouvé
            return chemin
    
    except Exception as e:
        # En cas d'erreur, renvoyer le message d'erreur
        return str(e)
    
    finally:
        # Fermeture du driver Neo4j
        driver.close()


In [25]:
def get_current_location():
    # Utilisez la fonction ip() pour obtenir l'emplacement basé sur votre adresse IP
    location = geocoder.ip('')
    return location.latlng  # Renvoie une liste contenant la latitude et la longitude

current_location = get_current_location()
if current_location:
    latitude, longitude = current_location

latitude,longitude

(34.261, -6.5802)

In [26]:
uri = "bolt://localhost:7687"
user = "neo4j"
password = "bdsas2024"
latitude = 34.016485
longitude = -5.033121

In [27]:
chemin = find_nearest_pharmacy(uri, user, password, latitude, longitude)

In [28]:
chemin

'|-5.0330673#34.0163942|-5.0332311#34.0163393|-5.0332311#34.0163393|-5.0334136#34.016677|-5.0335464#34.016629|-5.033541181730647#34.016630886121455|-5.0335195#34.0165709'

In [29]:
a = split_coordinates(chemin)

In [30]:
a = [(latitude,longitude)] + a

In [31]:
import folium

# Créer une carte centrée sur un emplacement spécifique
maps = folium.Map(location=a[0], zoom_start=50)

# Ajouter l'itinéraire à la carte
folium.Marker(location=a[0], icon=folium.Icon(color="green",icon='home')).add_to(maps)
folium.Marker(location=a[-1], icon=folium.Icon(color="red",icon='flag')).add_to(maps)
folium.PolyLine(a, color='blue', weight=5, opacity=0.5).add_to(maps)

# Afficher la carte
maps